In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random

import cv2
import numpy as np
import torch
import torchvision.transforms as tvtransforms

from pybbbc import BBBC021

import holoviews as hv

hv.extension('bokeh')

In [ ]:
bbbc021 = BBBC021()

In [ ]:
im, _ = bbbc021[4000]

In [ ]:
def normalize_image(im, new_min, new_max):
    im_max, im_min = im.max(), im.min()

    return (im - im_min) * (new_max - new_min) / (im_max - im_min) + new_min

In [ ]:
def adjust_gamma(im, gamma):
    
    # operate across channels independently

    gamma_ims = []

    for channel_idx in range(im.shape[2]):
        im_channel = im[..., channel_idx]

        im_max, im_min = im_channel.max(), im_channel.min()

#         gamma = np.random.uniform(gamma_low, gamma_high)

        im_channel = normalize_image(
            normalize_image(im_channel, 0, 1) ** gamma, im_min, im_max
        )

        gamma_ims.append(im_channel[..., np.newaxis])

    return np.concatenate(gamma_ims, axis=-1)

In [ ]:
ims = [adjust_gamma(im, gamma) for gamma in [0.5, 0.75, 1, 1.25, 1.5]]

In [ ]:
sigmas = [0, 0.025, 0.1, 0.2]

ims = [im + sigma * np.random.randn(*im.shape) for sigma in sigmas]

In [ ]:
hv.Layout([hv.RGB(im_.transpose(1, 2, 0), bounds=(0, 0, im.shape[2], im.shape[1]), label=f'{sigma=}') for im_, sigma in zip(ims, sigmas)]).cols(2)